In [18]:
import sqlite3
import pandas as pd

conn_netflix = sqlite3.connect('netflix.sqlite')
data = pd.read_sql("SELECT * FROM netflix_titles", conn_netflix)

table_actor_movie = pd.DataFrame(columns=['show_id', 'actor_id'])
table_actors = pd.DataFrame(columns=['actor_id', 'actor_name'])

actors_dict = {}
actor_id = 0
table_index = 0

for index, row in data.iterrows():
    cast = row['cast'].split(', ')
    show_id = row['show_id']

    for actor in cast:
        if actor not in actors_dict:
            actors_dict[actor] = actor_id
            table_actors.loc[actor_id] = [actor_id, actor]
            actor_id += 1

        actor_idx = actors_dict[actor]
        table_actor_movie.loc[table_index] = [show_id, actor_idx]
        table_index += 1

table_actors.to_sql('actors', conn_netflix, if_exists='replace', index=False)
table_actor_movie.to_sql('actor_movie', conn_netflix, if_exists='replace', index=False)

data.pop('cast')
data.to_sql('movies', conn_netflix, if_exists='replace', index=False)

df_test = pd.read_sql(
'''
SELECT a1.actor_name AS actor1, a2.actor_name AS actor2, COUNT(*) AS movie_count
FROM actor_movie am1
JOIN actor_movie am2 ON am1.show_id = am2.show_id
JOIN actors a1 ON am1.actor_id = a1.actor_id
JOIN actors a2 ON am2.actor_id = a2.actor_id
WHERE am1.actor_id < am2.actor_id
GROUP BY actor1, actor2
ORDER BY movie_count DESC
LIMIT 1;
''', conn_netflix)
print(df_test)

               actor1      actor2  movie_count
0  John Paul Tremblay  Robb Wells           15
